In [ ]:
import matplotlib.pyplot as plt

import os
import numpy as np

from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.applications.vgg16 import VGG16
import pickle
import tensorflow as tf


In [ ]:
model = VGG16(weights='imagenet', include_top=False)
model.summary()
c1 = model.layers[-16]
print(c1)

58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

In [ ]:
def save_all_features(nb_samples, source="images", dest="features", input_size = (416, 416), batch_size=6):
    
    # check if the directory exists, and if not make it
    if not os.path.exists(dest):
        os.makedirs(dest)
    
    # define image height and width
    (img_height, img_width) = input_size
    
    # build the VGG16 network and extract features after every MaxPool layer
    model = VGG16(weights='imagenet', include_top=False)
    
    c1 = model.layers[-16].output 
    c1 = GlobalAveragePooling2D()(c1)       

    c2 = model.layers[-13].output
    c2 = GlobalAveragePooling2D()(c2)       

    c3 = model.layers[-9].output
    c3 = GlobalAveragePooling2D()(c3)       

    c4 = model.layers[-5].output
    c4 = GlobalAveragePooling2D()(c4)       

    c5 = model.layers[-1].output
    c5 = GlobalAveragePooling2D()(c5)       

    model = Model(inputs=model.input, outputs=(c1,c2,c3,c4,c5))
    
    # define image generator without augmentation
    datagen = ImageDataGenerator(rescale=1./255.)    
    generator = datagen.flow_from_directory(
            source,
            target_size=(img_height, img_width),
            batch_size=batch_size,
            class_mode="sparse",
            shuffle=False)
    
    # generate and save features, labels and respective filenames
    steps = nb_samples/batch_size+1
    X = model.predict_generator(generator,steps)
    Y = np.concatenate([generator.next()[1] for i in range(0, generator.samples, batch_size)])
    names = generator.filenames

    for n, i in enumerate(X):
        with open(dest+"X-"+str(img_height)+"-c"+str(n+1)+"-AVG.npy", 'wb') as f:
                np.save(f, i)
    if not os.path.exists(dest+"Y.npy"):
        with open(dest+"Y.npy"  , 'wb') as f:
            np.save(f, Y)
    if not os.path.exists(dest+"filenames.npy"):
        with open(dest+"filenames.npy"  , 'wb') as f:
            np.save(f, names)

In [ ]:
def kfoldSVM_on_features(X, Y, modelname='0'):
    # define 10-fold cross validation test harness
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=555)
    cvscores, splits = [],[]
    for train, test in kfold.split(X, Y):
        clf = LinearSVC(C=1, loss='squared_hinge', penalty='l2',multi_class='ovr', max_iter=100000)
        clf.fit(X[train], Y[train])
        y_pred = clf.predict(X[test])
        acc = accuracy_score(Y[test],y_pred)*100
        cvscores.append(acc)
        splits.append((Y[test], y_pred))

        filename = ('finalized_model_' + str(modelname) + '.sav')
        pickle.dump(clf, open(filename, 'wb'))
    print("Accuracy score averaged across 10 kfolds %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    return np.mean(cvscores), np.std(cvscores)

In [ ]:
def evaluate(dest="features", size=416, strategy = "-AVG"):
    #
    size = str(size)
    l1 = np.load(dest+"X-"+size+"-c1"+strategy+".npy")
    l2 = np.load(dest+"X-"+size+"-c2"+strategy+".npy")
    l3 = np.load(dest+"X-"+size+"-c3"+strategy+".npy")
    l4 = np.load(dest+"X-"+size+"-c4"+strategy+".npy")
    l5 = np.load(dest+"X-"+size+"-c5"+strategy+".npy")
    a_all = np.concatenate([l1,l2,l3,l4,l5], 1)
    
    X =[l1, l2, l3, l4, l5, a_all]
    
    Y = np.load(dest +"Y.npy")
    acc_no_norm = []
    acc_norm = []
    x = a_all
    print("with square root normalization")
    x = np.sqrt(np.abs(x)) * np.sign(x)
    acc_norm.append(kfoldSVM_on_features(x, Y, modelname='with'))
    
    return acc_no_norm, acc_norm

In [ ]:
input_size = (416,416)
nb_samples = 10278
save_all_features(nb_samples, source="/content/datos/datos rede/train", dest="/content/features/", input_size=input_size, batch_size=50)

print()
print()
print("evaluating dataset with input size", input_size, "and GlobalAveragePooling2D")

acc_no_norm, acc_norm = evaluate(dest="./features/", size=input_size[0])

Found 10277 images belonging to 25 classes.
Instructions for updating:
Please use Model.predict, which supports generators.


evaluating dataset with input size (416, 416) and GlobalAveragePooling2D
with square root normalization


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
print(acc_no_norm)
print(acc_norm)

[(91.2841932841933, 2.461344239154871)]
[(93.19328419328419, 2.9440327289447317)]


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

image_path = '/content/datos/datos_reducido/val/Triturus marmoratus/https3A2F2Fobservation.org2Fphotos2F1414769.jpg'
# Open the image form working directory
image = Image.open(image_path)
data = np.asarray(image)
y = np.expand_dims(data, axis=0)
print(y.shape)
imgplot = plt.imshow(data)

In [ ]:
model = VGG16(weights='imagenet', include_top=False)
c1 = model.layers[-16].output 
c1 = GlobalAveragePooling2D()(c1)       
c2 = model.layers[-13].output
c2 = GlobalAveragePooling2D()(c2)       
c3 = model.layers[-9].output
c3 = GlobalAveragePooling2D()(c3)       
c4 = model.layers[-5].output
c4 = GlobalAveragePooling2D()(c4)       
c5 = model.layers[-1].output
c5 = GlobalAveragePooling2D()(c5)  
model = Model(inputs=model.input, outputs=(c1,c2,c3,c4,c5))
X = model.predict(y)

a_all = np.concatenate([X[0],X[1],X[2],X[3],X[4]], 1)



filename = '/content/finalized_model_5without.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(a_all)
print(result)

In [ ]:
a = np.load("/content/features/Y.npy")
b = np.load("/content/features/X-400-c1-AVG.npy")
c = np.load("/content/features/X-400-c2-AVG.npy")
files = np.load("/content/features/filenames.npy")

In [ ]:
print(a.shape)
print(b.shape)
print(c.shape)
print(files.shape)